In [19]:
# importing libraries
import tensorflow as tf
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import splitfolders
from tensorflow.keras.models import load_model

In [2]:
# split into train, test and val
# splitfolders.ratio("data/apple", output="data/apple", seed=1337, ratio=(.8, .1, .1), group_prefix=None)

In [3]:
# re-size all the images to this
IMAGE_SIZE = [224, 224, 3]

train_path = 'data/apple/train'
valid_path = 'data/apple/val'

In [4]:
# add preprocessing layer to the front of ResNet
keras_transfer = tf.keras.applications.MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=IMAGE_SIZE,
    classifier_activation="softmax"
)

# don't train existing weights
for layer in keras_transfer.layers:
    layer.trainable = False


In [5]:
# useful for getting number of classes
folders = glob('data/apple/train/*')
folders

['data/apple/train\\Apple_Apple_scab',
 'data/apple/train\\Apple_Black_rot',
 'data/apple/train\\Apple_Cedar_apple_rust',
 'data/apple/train\\Apple_healthy']

In [6]:
# our layers - you can add more if you want
x = Flatten()(keras_transfer.output)
prediction = Dense(len(folders), activation='softmax')(x)

In [7]:
# create a model object

model = Model(inputs=keras_transfer.input, outputs=prediction)

# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [8]:
# tell the model what cost and optimization method to use
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [9]:
train_datagen = ImageDataGenerator(rescale=1. / 224,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   rotation_range=0.2)

test_datagen = ImageDataGenerator(rescale=1. / 224)

training_set = train_datagen.flow_from_directory('data/apple/train',
                                                 target_size=(224, 224),
                                                 batch_size=32,
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('data/apple/val',
                                            target_size=(224, 224),
                                            batch_size=32,
                                            class_mode='categorical')

Found 4958 images belonging to 4 classes.
Found 618 images belonging to 4 classes.


In [10]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor="accuracy",
    patience=5,
    verbose=1,
    mode="auto",
)

In [11]:
# fit the model
apple_model = model.fit(
    training_set,
    validation_data=test_set,
    epochs=25,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set),
    callbacks=[callback],
    verbose=1
)

Epoch 1/25
155/155 [==============================] - 145s 924ms/step - loss: 0.9340 - accuracy: 0.9181 - val_loss: 0.4501 - val_accuracy: 0.9466
Epoch 2/25
155/155 [==============================] - 135s 873ms/step - loss: 0.2794 - accuracy: 0.9693 - val_loss: 0.2429 - val_accuracy: 0.9660
Epoch 3/25
155/155 [==============================] - 134s 861ms/step - loss: 0.2078 - accuracy: 0.9764 - val_loss: 0.1156 - val_accuracy: 0.9838
Epoch 4/25
155/155 [==============================] - 138s 892ms/step - loss: 0.2487 - accuracy: 0.9758 - val_loss: 0.1873 - val_accuracy: 0.9773
Epoch 5/25
155/155 [==============================] - 138s 891ms/step - loss: 0.1602 - accuracy: 0.9861 - val_loss: 0.4108 - val_accuracy: 0.9822
Epoch 6/25
155/155 [==============================] - 131s 844ms/step - loss: 0.1467 - accuracy: 0.9859 - val_loss: 0.2316 - val_accuracy: 0.9822
Epoch 7/25
155/155 [==============================] - 131s 844ms/step - loss: 0.2468 - accuracy: 0.9837 - val_loss: 0.1689 -

In [13]:
scores = model.evaluate(test_set)

20/20 [==============================] - 15s 738ms/step - loss: 0.1311 - accuracy: 0.9854


In [15]:
y_pred = model.predict(test_set)

20/20 [==============================] - 17s 787ms/step


In [16]:
y_pred

array([[0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 1.7104985e-38],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       ...,
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00]],
      dtype=float32)

In [17]:
y_pred = np.argmax(y_pred, axis=1)

In [18]:
y_pred

array([1, 3, 3, 0, 3, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 2, 1, 0, 1, 0, 0,
       0, 0, 2, 2, 0, 1, 0, 0, 1, 3, 2, 0, 2, 1, 3, 3, 1, 3, 3, 1, 0, 1,
       0, 0, 0, 1, 1, 3, 0, 1, 3, 0, 1, 1, 3, 2, 0, 2, 2, 2, 0, 0, 1, 2,
       0, 0, 0, 2, 3, 1, 2, 1, 3, 0, 0, 0, 3, 0, 1, 2, 3, 2, 3, 0, 2, 1,
       1, 1, 0, 3, 3, 3, 1, 3, 1, 1, 0, 1, 0, 3, 1, 3, 1, 2, 1, 3, 1, 3,
       0, 1, 0, 1, 3, 1, 3, 3, 0, 2, 3, 0, 0, 2, 1, 2, 2, 1, 3, 3, 2, 0,
       1, 0, 0, 0, 0, 0, 0, 3, 0, 1, 1, 1, 1, 2, 3, 0, 1, 0, 1, 1, 0, 0,
       3, 2, 3, 3, 0, 1, 3, 3, 1, 3, 2, 2, 1, 2, 1, 0, 0, 3, 1, 3, 1, 1,
       2, 1, 1, 1, 0, 0, 0, 0, 0, 3, 3, 2, 2, 1, 1, 1, 0, 3, 1, 1, 0, 1,
       0, 3, 0, 2, 1, 1, 0, 1, 3, 3, 1, 3, 0, 1, 0, 1, 0, 1, 3, 1, 0, 1,
       3, 0, 2, 0, 0, 1, 1, 3, 0, 3, 1, 2, 0, 2, 3, 0, 1, 3, 1, 1, 2, 2,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 2, 1, 0, 3, 3, 3, 0, 3, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 3, 1, 2, 0, 3, 0, 0, 3, 2, 3, 1, 0, 1, 3, 1, 1,
       0, 3, 2, 3, 0, 1, 1, 2, 0, 3, 2, 3, 0, 1, 2,

In [20]:
len(y_pred)

618

In [29]:
model.save('apple_model.h5')

In [30]:
model = load_model('apple_model.h5')

In [76]:
img = tf.keras.preprocessing.image.load_img('data/apple/test/Apple_Cedar_apple_rust/Apple_Cedar_apple_rust4820.jpg',
                                            target_size=(224, 224))

In [77]:
x = tf.keras.preprocessing.image.img_to_array(img)

In [78]:
x

array([[[166., 156., 155.],
        [164., 154., 153.],
        [162., 152., 151.],
        ...,
        [170., 165., 162.],
        [167., 162., 159.],
        [172., 167., 164.]],

       [[167., 157., 156.],
        [165., 155., 154.],
        [161., 151., 150.],
        ...,
        [168., 163., 160.],
        [165., 160., 157.],
        [171., 166., 163.]],

       [[169., 159., 158.],
        [166., 156., 155.],
        [161., 151., 150.],
        ...,
        [166., 161., 158.],
        [164., 159., 156.],
        [171., 166., 163.]],

       ...,

       [[108., 103., 100.],
        [106., 101.,  98.],
        [106., 101.,  98.],
        ...,
        [140., 135., 132.],
        [137., 132., 129.],
        [140., 135., 132.]],

       [[107., 102.,  99.],
        [105., 100.,  97.],
        [106., 101.,  98.],
        ...,
        [145., 140., 137.],
        [141., 136., 133.],
        [143., 138., 135.]],

       [[106., 101.,  98.],
        [104.,  99.,  96.],
        [106., 1

In [79]:
x.shape

(224, 224, 3)

In [80]:
x = x/255

In [81]:
x = np.array([x])

In [82]:
pred = model.predict(x)

1/1 [==============================] - 0s 63ms/step


In [83]:
pred

array([[0., 0., 1., 0.]], dtype=float32)

In [84]:
a = np.argmax(pred, axis=1)

In [85]:
a

array([2], dtype=int64)

In [2]:
import tensorflow as tf
dir_name_list = ['apple', 'cherry', 'corn', 'grape', 'pepper_bell', 'potato', 'strawberry', 'tea', 'tomato']
for i in dir_name_list:
    model1 = tf.keras.models.load_model(f'trained_model/{i}.h5')
    converter = tf.lite.TFLiteConverter.from_keras_model(model1)
    tflite_model = converter.convert()
    open(f"trained_model/tflite_models/{i}_lite.tflite", "wb").write(tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmpmw08y24q/assets


INFO:tensorflow:Assets written to: /tmp/tmpmw08y24q/assets
2022-10-20 20:18:23.809798: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-10-20 20:18:23.809833: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-10-20 20:18:23.809982: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpmw08y24q
2022-10-20 20:18:23.831436: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-10-20 20:18:23.831476: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpmw08y24q
2022-10-20 20:18:23.913994: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-10-20 20:18:24.319300: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpmw08y24q
2022-10-20 20:18:24.445387: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

INFO:tensorflow:Assets written to: /tmp/tmpi467iuiv/assets


INFO:tensorflow:Assets written to: /tmp/tmpi467iuiv/assets
2022-10-20 20:18:42.439813: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-10-20 20:18:42.439844: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-10-20 20:18:42.439975: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpi467iuiv
2022-10-20 20:18:42.455950: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-10-20 20:18:42.455984: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpi467iuiv
2022-10-20 20:18:42.520966: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-10-20 20:18:42.832612: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpi467iuiv
2022-10-20 20:18:42.934647: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

INFO:tensorflow:Assets written to: /tmp/tmps020wmwn/assets


...

INFO:tensorflow:Assets written to: /tmp/tmp10dfzxko/assets
2022-10-20 20:20:18.086651: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-10-20 20:20:18.086684: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-10-20 20:20:18.086848: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp10dfzxko
2022-10-20 20:20:18.108039: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-10-20 20:20:18.108079: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp10dfzxko
2022-10-20 20:20:18.188396: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-10-20 20:20:18.585465: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmp10dfzxko
2022-10-20 20:20:18.732908: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

INFO:tensorflow:Assets written to: /tmp/tmpqk6svtb9/assets


INFO:tensorflow:Assets written to: /tmp/tmpqk6svtb9/assets
2022-10-20 20:20:36.363380: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-10-20 20:20:36.363416: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-10-20 20:20:36.363581: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpqk6svtb9
2022-10-20 20:20:36.398119: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-10-20 20:20:36.398160: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpqk6svtb9
2022-10-20 20:20:36.494243: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-10-20 20:20:36.904575: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpqk6svtb9
2022-10-20 20:20:37.030496: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

INFO:tensorflow:Assets written to: /tmp/tmpzb0ip5y2/assets


INFO:tensorflow:Assets written to: /tmp/tmpzb0ip5y2/assets
2022-10-20 20:20:56.311652: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-10-20 20:20:56.311683: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-10-20 20:20:56.311824: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpzb0ip5y2
2022-10-20 20:20:56.329433: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-10-20 20:20:56.329470: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpzb0ip5y2
2022-10-20 20:20:56.395859: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-10-20 20:20:56.753305: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpzb0ip5y2
2022-10-20 20:20:56.862703: I tensorflow/cc/saved_model/loader.cc:305] SavedModel